In [2]:
import gym, gym_unbalanced_disk, time
import numpy as np
from stable_baselines3 import PPO, SAC
import matplotlib.pyplot as plt
from dataclasses import dataclass

In [3]:
def normalise_angle(theta):
    return ((theta)%(2*np.pi))-np.pi

def compute_reward(self):
    q = normalise_angle(self.th)
    theta=self.th
    omega = self.omega
    u = self.u

    r=-100*q**2 
    return r

In [4]:
target_angle=np.pi
angle_reward = lambda self: ((np.cos(self.th-target_angle)+1.5)**2 - 0.25)
correctVelocity_reward = lambda self: 0.00125*(((np.cos(self.th)+1)/2)*(self.omega)**2)
voltage_penalty = lambda self: -0.01*((self.u)**2)
incorrectAngle_penalty = lambda self: -0.01*np.abs(self.th-target_angle)

reward_function = lambda self: angle_reward(self) + correctVelocity_reward(self) + voltage_penalty(self) + incorrectAngle_penalty(self)

In [16]:

env = gym.make('unbalanced-disk-v0', dt=0.025, umax=3., reward_fun=reward_function) 

# model = PPO('MlpPolicy', env, verbose=1) #b)
model = SAC('MlpPolicy', env, verbose=2, learning_starts=1000) #b)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [17]:
model=model.load('SAC_working_save.zip', env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [18]:
env.close()
env = gym.make('unbalanced-disk-exp-v0', dt=0.025, umax=3.)  # gym.make('unbalanced-disk-v0', dt=0.025, umax=3.,reward_fun=compute_reward,noise_scale=1e-3) 
render_sim = True
obs = env.reset()
u_list=[]
thetas=[]
omegas=[]
rewards=[]
try:
    for _ in range(1000):
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        if render_sim:
            env.render()
        u_list.append(action)
        thetas.append(obs[0])
        omegas.append(obs[1])
        rewards.append(reward)
        if render_sim:
            time.sleep(1/50)
        if done:
            obs=env.reset()
finally:
    env.close()

KeyboardInterrupt: 

In [8]:
def get_color(q):
    if abs(q) > 2 * np.pi / 3:  # lower 120
        c=0
    elif abs(q) > np.pi / 3:  # either side
        c=1
    else:  # top
        c=2
    return c

def compute_reward_sim(self):
    q = normalise_angle(self.th)
    r=compute_reward(self)
    c=get_color(q)
    return q, r, c

In [9]:
@dataclass
class Data:
    th = None
    omega = None
    u = None

In [10]:
rewards_sim=[]
colors=[]
thetas_norm=[]
colormap = np.array(['r', 'g', 'b'])
for i in range(len(u_list)):
    data=Data()
    data.th=thetas[i]
    data.omega=omegas[i]
    data.u=u_list[i]
    q, r,c = compute_reward_sim(data, )
    rewards_sim.append(r)
    colors.append(c)
    thetas_norm.append(q)

rewards_sim=np.array(rewards_sim)
rewards_sim=rewards_sim
colormap = np.array(['r', 'g', 'b'])


plot_start=0#np.argmax(rewards)
plot_end=plot_start+50

plt.figure(figsize=(15,15))
[plt.axhline(x, ls='--',c='black',alpha=0.4) for x in np.arange(-1,1.1,1/3)*np.pi]

plt.plot(thetas[plot_start:plot_end], label='theta')
plt.plot(thetas_norm[plot_start:plot_end], label='theta norm')
plt.plot(omegas[plot_start:plot_end], label='omega')
# plt.plot(np.array(omegas[:plot_end])**2, label='omega^2')
plt.plot(u_list[plot_start:plot_end], label='u list')
plt.plot(rewards[plot_start:plot_end], linewidth=2, label='reward')
plt.scatter([*range(len(u_list[plot_start:plot_end]))], rewards_sim[plot_start:plot_end], ls='--', linewidth=2, label='reward sim', c=colormap[colors[plot_start:plot_end]])
plt.legend()
plt.show()